# Bayesian Regression

Bayesian Resgression with rookies filtered out.

In [23]:
import pymc3 as pm
import pandas as pd
import numpy as np
import arviz as az

data = pd.read_csv("../data/shifts_data_final_2018_19.csv")
data.drop(data.columns[0], axis = 1, inplace = True)
data.head()

,point_diff_per_100,home_team,away_team,0,1,2,3,4,5,6,...,519,520,521,522,523,524,525,526,527,528
0,-36.458333,Celtics,Nuggets,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39.062500,Celtics,Nuggets,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-72.337963,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-36.168981,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.296569,Celtics,Nuggets,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Note - the priors are too small. Scale them up to range from like -10 to 10

So far with standard dev of 4, the results are worse with these priors compared to the old priors we used before. We see random players with very high +/-

In [24]:
rookie_priors_df = pd.read_csv("../data/priors_rookies.csv")
rookie_priors_df.columns = ["index", "mu", "sd", "name", "type"]
rookie_priors_df

rookie_scale = 11.207822/max(rookie_priors_df.mu)

rookie_scale

rookie_priors_df.mu *= rookie_scale

rookie_priors_df

,index,mu,sd,name,type
0,1,9.005090,3,Jayson Tatum,Rookie
1,2,6.947821,3,Jaylen Brown,Rookie
2,3,4.099367,3,Terry Rozier,Rookie
3,14,2.624281,3,OG Anunoby,Rookie
4,16,3.409294,3,Delon Wright,Rookie
...,...,...,...,...,...
97,494,3.395777,3,Dario Saric,Rookie
98,504,2.569450,3,Aaron Holiday,Rookie
99,507,3.167747,3,Lonnie Walker IV,Rookie
100,514,7.736895,3,Brandon Ingram,Rookie


In [25]:
# store home and away teams for potential use later when we incorporate team ratings
home_teams = data['home_team']
away_teams = data['away_team']
# now drop these columns from the main training dataframe
data.drop(['home_team', 'away_team'], axis = 1, inplace = True)
data.head()

,point_diff_per_100,0,1,2,3,4,5,6,7,8,...,519,520,521,522,523,524,525,526,527,528
0,-36.458333,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39.062500,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-72.337963,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-36.168981,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.296569,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
prior_sd4 = np.full(101, 4)
prior_sd3 = np.full(101, 3)
prior_sd2 = np.full(101, 2)


idx_set = set(rookie_priors_df["index"].astype(str))

In [44]:
# need to rename columns now since numbers confuse pymc3

cols = [col for col in data.columns if col in idx_set or col == "point_diff_per_100"]

rookie_data = data[cols]
#rookie_data
rookie_data.head()

new_cols = []
for i in range(np.shape(rookie_data)[1]):
    if i == 0:
        new_cols.append("point_diff")
    else:
        new_cols.append("p" + rookie_data.columns[i])

x_df = rookie_data
x_df.columns = new_cols
x_df.head()


,point_diff,p1,p2,p3,p14,p16,p18,p24,p26,p33,...,p448,p458,p459,p472,p481,p494,p504,p507,p514,p515
0,-36.458333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39.062500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-72.337963,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-36.168981,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,38.296569,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x = np.array(x_df.iloc[:,1:])
y = np.array(x_df.iloc[:,0])

x_shape = np.shape(x)[1]
    
with pm.Model() as model4:
    # priors
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = rookie_priors_df.mu # defined above
    x_prior_sigmas = rookie_priors_df.sd # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    trace4 = pm.sample(1000, tune = 1000, cores = 1)
    
    

<ipython-input-46-c0d3db42793b>:17: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace4 = pm.sample(1000, tune = 1000, cores = 1)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [x, Intercept, sigma]


In [ ]:
with pm.Model() as model3:
    # priors
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = prior_means # defined above
    x_prior_sigmas = prior_sd3 # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    trace3 = pm.sample(1000, tune = 1000, cores = 1)

In [ ]:
with pm.Model() as model2:
    # priors
    sigma = pm.HalfCauchy("sigma", beta=10) # arbitrarily defined
    intercept = pm.Normal("Intercept", 0, sigma=20) # arbitrarily defined
    x_prior_means = prior_means # defined above
    x_prior_sigmas = prior_sd2 # defined above
#     x_prior_means = np.zeros(x_shape) # just testing with mean zero to compare to ridge
    x_coeff = pm.Normal("x", mu = x_prior_means, sigma=x_prior_sigmas, shape = x_shape) # original method - no list comprehension

    likelihood = pm.Normal("y", mu=intercept + x_coeff.dot(x.T), sigma=sigma, observed=y) # original method - no list comprehension
    
    trace2 = pm.sample(1000, tune = 1000, cores = 1)

In [ ]:
with model4:
    results_df4 = az.summary(trace4)
    
with model3:
    results_df3 = az.summary(trace3)
    
# with model2:
#     results_df2 = az.summary(trace2)

In [ ]:
# import the player map dictionary to go between index, player_id, and name
player_index_map = pd.read_csv("../data/player_index_map.csv")

In [ ]:
player_index_map.loc[player_index_map.index == 274]

In [ ]:
print((results_df4.loc[results_df4['mean'] > 6]).sort_values(by=['mean']))

In [ ]:
print((results_df3.loc[results_df3['mean'] > 5]).sort_values(by=['mean']))

In [ ]:
print((results_df2.loc[results_df2['mean'] > 5]).sort_values(by=['mean']))

## The following cell shows how to get the distribution for a specific player

In this case we get the distribution for Steph Curry (index 317)

In [ ]:
import seaborn as sns

with model3:
#     az.plot_trace(trace3)
#     print(np.shape(trace3['x']))
#     print(max(trace3['x'][:,317]))
    sns.distplot(trace3['x'][:,317], hist = False)

In [ ]:
with model3:
    az.plot_trace(trace3)